In [9]:
from PIL import Image

im = Image.open('./src/assets/sprites/warlock_char.png').convert('RGBA')

# Get all pixels as a list of tuples (R, G, B, A)
pixels = list(im.getdata())

# For each pixel: if alpha is not zero, set to 255, otherwise keep as-is
new_pixels = [
    (r, g, b, 255) if a != 0 else (r, g, b, 0)
    for (r, g, b, a) in pixels
]

im.putdata(new_pixels)

# Save the result (you can overwrite or use a new file)
im.save('./src/assets/sprites/output.png')


In [10]:
import math



def color_distance(c1, c2):
    # Euclidean distance in RGB space
    return math.sqrt(sum((a - b) ** 2 for a, b in zip(c1, c2)))

# Parameters
target_color = (44, 39, 53)      # #2c2735
replacement_color = (12, 5, 12)  # #0c050c
threshold = 35                   # Adjust this for more/less strictness

# Load image
im = Image.open('./src/assets/sprites/output.png').convert('RGBA')
pixels = list(im.getdata())

new_pixels = []
for r, g, b, a in pixels:
    if color_distance((r, g, b), target_color) <= threshold:
        new_pixels.append((*replacement_color, a))
    else:
        new_pixels.append((r, g, b, a))

im.putdata(new_pixels)
im.save('./src/assets/sprites/output2.png')

In [11]:
import json

# Load your JSON data from file or directly assign here
with open('chat_history.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

output_lines = []

# Loop over each conversation
for conversation_id, messages in data.items():
    for idx, msg in enumerate(messages):
        content = msg.get("Content")
        if content and "album" in content.lower():
            # Get the 10 before, 1 current, 10 after
            start = max(0, idx - 10)
            end = min(len(messages), idx + 11)

            snippet = messages[start:end]
            for entry in snippet:
                who = entry.get("From", "Unknown")
                created = entry.get("Created", "")
                text = entry.get("Content", "[No Content]")
                output_lines.append(f"{who} ({created}): {text}")
            output_lines.append("")  # One empty line
            output_lines.append("")  # Second empty line

# Write output to file
with open("album_contexts.txt", "w", encoding='utf-8') as f:
    f.write("\n".join(output_lines))
